## Ensure that you have relevant data in data_untracked

In [1]:
import numpy as np
import pandas as pd
import sys
import os



parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from create_labels import labels
from extract_data import extract 
from create_features import features
from preprocess_feature import preprocess
# from create_data import create_features

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

AttributeError: module 'create_features.features' has no attribute 'GRAPH_FEATURE'

### Download below required if creating footnote analysis file

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download("wordnet")
# nltk.download("averaged_perceptron_tagger")

___
## Section 1. Create Data
___

### Extract form 4 data if missing

In [ ]:
data_extractor = extract.Data_Extractor()
data_extractor.create_form4()

### Create labels

In [ ]:
# Create data labels if missing
label_data_creator =  labels.Label_Data_Creator()
label_data_creator.create_labels()

========== Required labelled files present ==========
========== Skip data label creation ==========
========== Reading successful ==========


### Create features

In [ ]:
feature_data_creator = features.Feature_Data_Creator()
feature_data_creator.create_features()

=== Final features file not found. Begin creating ===
=== Transaction Key file is found. Extracting ===
=== Footnote Key file is found. Extracting ===
=== Graph Key file is found. Extracting ===
=== Network time_independent_features file is found. Extracting ===
=== Network time_dependent_features file is found. Extracting ===
=== Other Features Key file is found. Extracting ===
=== Saving file ===


### Create training and testing data

In [ ]:
feature_preprocessor = preprocess.Feature_Preprocessor()
feature_preprocessor.extract()
feature_preprocessor.create_training_testing()

=== Training or Testing file not found. Begin creating based on quantile: 0.7
=== Saving Training and Testing ===


In [ ]:
feature_preprocessor_baseline = preprocess.Feature_Preprocessor()
feature_preprocessor_baseline.extract(["TRANS_CODE"])
feature_preprocessor.baseline_create_training_testing()

___
## Section 2. Modelling, Training and Validation
___

In [21]:
print("features:")
print(feature_data_creator.features)

features:
['js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyist_score_final', 'total_senate_connections', 'total_house_connections', 'combined_seniority_score', 'PI_combined_total', 'net_trading_intensity', 'net_trading_amt', 'relative_trade_size_to_self', 'relative_trade_size_to_others', 'beneficial_ownership_score', 'js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyist_score_final', 'total_senate_connections', 'total_house_connections', 'combined_seniority_score', 'PI_combined_total', 'net_trading_intensity', 'net_trading_amt', 'relative_trade_size_to_self', 'relative_trade_size_to_others', 'beneficial_ownership_score', 'js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyi

___
## Compare results
___

In [8]:
data =pd.read_csv("../../data_untracked/processed/testing_full_features.csv")
list(data.columns)

['TRANS_SK',
 'ACCESSION_NUMBER',
 'TRANS_DATE',
 'ISSUERTRADINGSYMBOL',
 'RPTOWNERNAME_;',
 'snorkel_prob',
 'y_pred',
 'ppr_topK_exp',
 'num_topK_neighbors',
 'ppr_house_0.85',
 'ppr_house_0.95',
 'ppr_senate_0.85',
 'ppr_senate_0.95',
 'NODEID',
 'important_connections',
 'full_congress_connections',
 'house_t2_important_connections',
 'house_t2_full_congress_connections',
 'house_t1_important_connections',
 'house_t1_full_congress_connections',
 'sen_important_connections',
 'sen_full_congress_connections',
 'sen_t2_important_connections',
 'sen_t2_full_congress_connections',
 'sen_t1_important_connections',
 'sen_t1_full_congress_connections',
 'full_congress_connections_z',
 'sen_full_congress_connections_z',
 'sen_t2_full_congress_connections_z',
 'house_t2_full_congress_connections_z',
 'sen_important_connections_z',
 'sen_t2_important_connections_z',
 'important_connections_z',
 'house_t2_important_connections_z',
 'full_congress_connections_z_is_low',
 'full_congress_connecti

In [17]:
pd.set_option('display.max_columns', None)
data.describe()

,TRANS_SK,snorkel_prob,y_pred,ppr_topK_exp,num_topK_neighbors,ppr_house_0.85,ppr_house_0.95,ppr_senate_0.85,ppr_senate_0.95,NODEID,important_connections,full_congress_connections,house_t2_important_connections,house_t2_full_congress_connections,house_t1_important_connections,house_t1_full_congress_connections,sen_important_connections,sen_full_congress_connections,sen_t2_important_connections,sen_t2_full_congress_connections,sen_t1_important_connections,sen_t1_full_congress_connections,full_congress_connections_z,sen_full_congress_connections_z,sen_t2_full_congress_connections_z,house_t2_full_congress_connections_z,sen_important_connections_z,sen_t2_important_connections_z,important_connections_z,house_t2_important_connections_z,full_congress_connections_z_is_low,full_congress_connections_z_is_high,sen_full_congress_connections_z_is_low,sen_full_congress_connections_z_is_high,sen_t2_full_congress_connections_z_is_low,sen_t2_full_congress_connections_z_is_high,house_t2_full_congress_connections_z_is_low,house_t2_full_congress_connections_z_is_high,sen_important_connections_z_is_low,sen_important_connections_z_is_high,sen_t2_important_connections_z_is_low,sen_t2_important_connections_z_is_high,important_connections_z_is_low,important_connections_z_is_high,house_t2_important_connections_z_is_low,house_t2_important_connections_z_is_high,js_bin,s_bin,b_bin,jb_bin,ob_bin,gb_bin,gift,distribution,charity,price,number,ball,pursuant,10b5-1,16b-3,net_trading_intensity,net_trading_amt,relative_trade_size_to_self,beneficial_ownership_score,title_score,filing_lag_days,is_lobby_True,has_lobby_give,has_lobby_give_and_receive,has_lobby_receive,has_donate_give,has_donate_give_and_receive,has_donate_receive,full_congress_connections_z_cat_high,full_congress_connections_z_cat_low,full_congress_connections_z_cat_medium,sen_full_congress_connections_z_cat_high,sen_full_congress_connections_z_cat_low,sen_full_congress_connections_z_cat_medium,sen_t2_full_congress_connections_z_cat_high,sen_t2_full_congress_connections_z_cat_low,sen_t2_full_congress_connections_z_cat_medium,house_t2_full_congress_connections_z_cat_high,house_t2_full_congress_connections_z_cat_low,house_t2_full_congress_connections_z_cat_medium,sen_important_connections_z_cat_high,sen_important_connections_z_cat_low,sen_important_connections_z_cat_medium,sen_t2_important_connections_z_cat_high,sen_t2_important_connections_z_cat_low,sen_t2_important_connections_z_cat_medium,important_connections_z_cat_high,important_connections_z_cat_low,important_connections_z_cat_medium,house_t2_important_connections_z_cat_high,house_t2_important_connections_z_cat_low,house_t2_important_connections_z_cat_medium,TRANS_TIMELINESS_clean_early,TRANS_TIMELINESS_clean_late,TRANS_TIMELINESS_clean_on_time,execution_timeliness_after_trans,execution_timeliness_before_trans,execution_timeliness_on_trans,filing_timeliness_early_filing,filing_timeliness_late_filing,filing_timeliness_on_time_filing
count,3.470200e+05,347020.000000,347020.000000,3.470200e+05,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,3.470200e+05,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.000000,347020.00